In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/data.csv', delimiter=';', decimal=",")

# Преобразование строки в periodStartDate и periodEndDate
df["periodStartDate"] = pd.to_datetime(df["periodStartDate"], format="%d.%m.%Y %H:%M")
df["periodEndDate"] = pd.to_datetime(df["periodEndDate"], format="%d.%m.%Y %H:%M")
# Преобразование periodDuration в секунды
df["periodDuration"] = pd.to_timedelta(df["periodDuration"]).dt.total_seconds()


In [ ]:
#RNN
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Выбор признаков и целевых переменных
features = df[[
    'minCupolaTemp', 'avgCupolaTemp', 'cupolaTempChangeRate',
    'minSmokeTemp', 'smokeTempChangeRate',
    'minGasFlowRate', 'maxGasFlowRate', 'avgGasFlowRate', 'sumGasFlowRate'
]]
targets = df[['maxCupolaTemp', 'maxSmokeTemp', 'periodDuration']]

# Нормализация данных
scaler_features = StandardScaler()
scaler_targets = StandardScaler()

X = scaler_features.fit_transform(features)
y = scaler_targets.fit_transform(targets)

# Преобразование в 3D-формат для RNN (samples, timesteps, features)
X = X.reshape(X.shape[0], 1, X.shape[1])

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Создание RNN-модели
model = Sequential([
    SimpleRNN(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    SimpleRNN(32),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(3)  # 3 выходных значения: maxCupolaTemp, maxSmokeTemp, periodDuration
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

model.summary()

# 3. Обучение модели
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# 4. Визуализация процесса обучения
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('RNN Model Loss Progress')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# 5. Оценка модели
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE (RNN): {test_mae:.4f}")

# 6. Пример прогнозирования
y_pred = model.predict(X_test)
y_pred_original = scaler_targets.inverse_transform(y_pred)
y_test_original = scaler_targets.inverse_transform(y_test)

# Вывод первых 5 предсказаний
print("\nПримеры предсказаний:")
for i in range(5):
    print(f"\nОбразец {i+1}:")
    print(f"Предсказано: {y_pred_original[i]}")
    print(f"Фактически:  {y_test_original[i]}")

# Выбор целевой переменной для сравнения (например, maxCupolaTemp)
# Предположим, что y_test_original[:, 0] соответствует maxCupolaTemp
plt.figure(figsize=(10, 6))

# График рассеяния
plt.scatter(y_test_original[:, 0], y_pred_original[:, 0], alpha=0.6, label='Предсказанные значения')
plt.plot([y_test_original[:, 0].min(), y_test_original[:, 0].max()],
         [y_test_original[:, 0].min(), y_test_original[:, 0].max()],
         'r--', lw=2)  # Диагональная линия

plt.title('Сравнение предсказанных и реальных значений')
plt.xlabel('Реальные значения')
plt.ylabel('Предсказанные значения')
plt.grid()
plt.show()